# (Auto) Regression Analysis in Python

In this session we are going to fit a regression model to predict daily mean air temperature. The approach we will use is technically known as "auto" regression because our predictor variables will be weather elements (including air temperature itself) observed at earlier time periods (in this case, the day before). 

To complete the regression analysis, you will be using some custom-made functions I have created for you. These are sitting in a file called "tom.py" -- visible on this session's Learn page. You should download this file to *the same directory* as this notebook; only if they are in this directory will you be able to import the module and use its code. 

The module I have written will do a few things for you, including: 

- (1) Calculating the day of year climatologies for the different meteorological variables we want to use to predict air temperature. The climatology is also (re-) computed for air temperature -- the 'predictand'. Note that I say 're-computed' because we already computed the day-of-year climatology for air temperature because we already calculated this in earlier sessions
- (2) Subtracting these day of year climatologies from the raw series to leave the anomalies (i.e. the weather) for each predictor variable and for the predictand 
- (3) Fits a regression of the form:

\begin{equation*}
Y_t = \alpha + \sum_{i=1}^{i=n_v}{\beta_i \times X_{i,t-k}}
\end{equation*}

in which *i* is indexing the predictor variable, and *t* is indexing the date. The $\beta$ terms are the 'slope' regression coefficients, identifying the change in mean temperature on day $t$ per unit change in the predictor variable $X_{i}$ on day $t - k$ (where $k$ is the 'lag' -- the number of days separating the predictors and the predictor).

In words, then, our modelling approach is asking if the *weather* $k$ days in the past was relatively warm/cold/windy/still... how much warmer or colder than 'normal' will it be today? 

The codes I have written for you will then:

- (4) Assess which variables in the regression are *statistically significant* -- at a user-specified *p*-value 
- (5) Re-fit the regression using $only$ those statistically-significant variables
- (7) Add the temperature climatology to the regression-modelled anomaly 

Appreciating the nuance of step (7) is important. The regression is modelling how much warmer/colder than normal it will be; we then need to add the 'normal' (i.e. the *climatology*) back in to predict the actual temperature. 

Helpfully, the code finally: 
- (8) Returns this simulated (and observed) temperature for a specific year -- which we can choose -- and which is used to evaluate the model's performance. [Note: data from this year are *not* used to fit the model] 
- (9) Calculates and returns the R squared of the 'final' model. The R squared is just the correlation coefficient, squared. It is used to indicate how closely the observed and modelled series *co-vary* and is therefore a useful diagnostic; however, it *does not* replace the $\it{MAE}$ as our performance (error) metric of choice.


If all this sounds complicated, fear not. Doing all of the above is achieved with a single call to the auto_regress tom function. Run the code below to import the relevant modules ready to run the analysis

In [2]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tom

Now we are almost ready to begin working. In the code below notice that I import the original campus meteorological data using the read_csv method of Pandas. You should already have this met file stored somewhere on your computer (we worked with it in previous sessions,and I have made it available on Learn for this week), and should now be comfortable using Pandas to import data.  

Run the code below to load in these data and compute the daily means. 

In [3]:
fin="C:/Users/gytm3/OneDrive - Loughborough University/Teaching/GYP035/201920/StudentData/Campus_Met_Data_Student.csv"
date_parser=lambda x: pd.datetime.strptime(x,"%d/%m/%Y %H:%M")
#obs=pd.read_csv(fin,parse_dates=["TIMESTAMP"],date_parser=date_parser,index_col=0)
obs=pd.read_csv(fin,parse_dates=True,index_col=0)
obs=obs.resample("D").mean()
obs.head(50)

,RECORD,AirTC_Avg,RH,T107_C,T107_C_2,WP_kPa,WP_kPa_2,VW_Avg,WS_ms_S_WVT,WindDir_D1_WVT,WindDir_SD1_WVT,SlrkW_Avg,SlrMJ_Tot,NR_Wm2_Avg,CNR_Wm2_Avg,Rain_mm_Tot,BP_mbar_Avg
TIMESTAMP,,,,,,,,,,,,,,,,,
2016-04-08,95.5,9.283875,78.483750,-124.667391,-136.122340,-3.693136,-3.691800,NaN,1.172115,214.574375,30.691354,0.114344,1.029706,50.623448,50.623448,0.0,1003.010417
2016-04-09,191.5,7.281344,79.211563,-130.942391,-130.373913,-3.693846,-3.695897,NaN,1.355052,226.177188,34.652396,0.118104,1.063606,41.474750,41.474229,0.0,994.593750
2016-04-10,287.5,7.002740,72.707812,-110.322917,-116.109677,-3.694404,-3.693700,NaN,1.763354,118.349375,26.402500,0.167385,1.507092,69.246813,69.265750,0.0,1000.677083
2016-04-11,383.5,10.468208,83.864583,-122.585106,-136.090526,-3.696491,-3.697378,NaN,2.170979,63.713542,24.319479,0.111875,1.007265,49.099260,49.183844,0.0,1000.250000
2016-04-12,479.5,10.058594,84.848438,-122.554348,-133.010753,-3.692864,-3.694217,NaN,0.971427,241.214979,32.682708,0.077865,0.701586,29.171917,29.171917,0.0,998.697917
2016-04-13,575.5,10.684615,73.719479,-108.994624,-120.703191,-3.695511,-3.697204,NaN,0.864771,212.739323,44.977396,0.199010,1.791786,88.577031,88.577031,0.0,1000.927083
2016-04-14,671.5,9.619125,84.003125,-117.427083,-118.176344,-3.693333,-3.691795,NaN,1.730604,68.041333,28.710938,0.116302,1.047325,53.505448,53.507635,0.0,1002.906250
2016-04-15,767.5,6.991844,91.731250,-119.930851,-118.111702,NaN,NaN,NaN,1.542917,104.314375,25.046250,0.034750,0.313794,1.408844,1.408198,0.0,995.135417
2016-04-16,863.5,4.890479,78.952917,-116.613542,-132.361053,-3.695833,-3.685250,NaN,1.782406,279.082500,31.969167,0.152094,1.368960,61.767760,61.780260,0.0,997.510417


The *auto_regress* method of the tom module calibrates and applies the regression. It has the following call-signature:

predicted, r$^{2}$ = auto_regress(target,predictors,year_predict,days_behind,p_sig,verbose). 

We can understand what the terms are by using the code 'help(tom.auto_regress)'. Try it below to get an explanation of the different input/output variables: 

In [4]:
help(tom.auto_regress)

Help on function auto_regress in module tom:

auto_regress(target, predictors, year_predict, days_behind, p_sig, verbose=True)
    This function takes a Pandas Series (target)
    of weather and fits a multiple regression using 
    the predictors -- observations of other weather 
    variables made days_nehind. 
    
    Before the regression is fitted, the target and
    predictors have the seasonal cycle removed, so 
    they are 'anomalies' relative to the climatology
    
    Only predictors with a p-value <= p_sig are 
    included in the model. 
    
    Inputs:
    
        - target       : Pandas Series - variable 
                         we want to predict
        - predictors   : Pandas DataFrame - observations 
                         of all weather variables to 
                         possibly include in the (auto)
                         regression
        - year_predict : Int - the year we want to evaluate 
                         model perfoemance on. This year
  

Below I call auto_regress using only air temperature and relative humdidity as the candidate predictor variables; using all but 2019's data to fit the model (and holding back 2019 to test model performance); a lag of 1; a threhold *p*-value of 0.05; and we ask the function to print out some summary text.

Run this code to get a feel for how the model works. 

In [5]:
pred,r2 = tom.auto_regress(obs["AirTC_Avg"],obs[["AirTC_Avg","RH"]],2019,1,0.05,True)

-----------------------------------------------
Fitted model with all terms. Diagnostics follow
-----------------------------------------------
P-value for constant (intercept) = 0.9607
P-value for variable: AirTC_Avg = 0.000
(...Including this variable in model...)
P-value for variable: RH = 0.019
(...Including this variable in model...)
-----------------------------------------------


*      *      *     *      *      *

Modelling complete!
Final model:
	AirTC_Avg(t) = 0.733 x AirTC_Avg(t-1) + 0.022 x RH(t-1)

*      *      *     *      *      *


The ouptut tells us that both temperature and relative humidity on the previous day are significant predictors of temperature on the current day. The intercept term is not significantly different from zero (at *p* = 0.05), so it was automatically omitted from the *final* model. 

Check your understanding: according to this regression model, if the air temperature was 1$^{\circ}$C warmer than normal yesterday, and relative humidity was also 10% above normal on that day, what temperature $anomaly$ should we expect today? You do not need to do any programming to answer this question! Have your understanding checked by one of the staff before moving on.

The transparency of regression models is one of their major attractions. Once we learn how to interpret the meaning of the coefficients' values, we can readily learn the direction in which the different variables work, and can see how important they are. 

So, how good can we get our model? Probably better than it is at present with just the two predictors (temperature and relative humidity) used so far. I want you to try fitting a more comprehensive model now. To remind yourself of the variables available in the campus met file, run the following code. 

In [6]:
for i in obs.columns: print(i)

RECORD
AirTC_Avg
RH
T107_C
T107_C_2
WP_kPa
WP_kPa_2
VW_Avg
WS_ms_S_WVT
WindDir_D1_WVT
WindDir_SD1_WVT
SlrkW_Avg
SlrMJ_Tot
NR_Wm2_Avg
CNR_Wm2_Avg
Rain_mm_Tot
BP_mbar_Avg


Not all of these will be suitable for use in linear regression without more careful thought (because, for example, they occupy a circular scale [wind direction] or because they have distributions which are not even close to normal [rain]). For now, I will constrain your options. You should only consider using the following as candidate regression predictors: 

  - AirTC_Avg    : familiar to you already -- daily mean air tempature ($^{\circ}$C)
  - RH           : daily mean relative humidity (0-->100%)
  - WS_ms_S_WVT  : daily mean wind speed (m s$^{-1}$)
  - SlrkW_Avg    : daily mean incoming shortwave (i.e. solar) radiation (kW m$^{-2}$)
  - NR_Wm2_Avg   : daily mean *net* radiation (solar and longwave)
  - BP_mbar_Avg  : daily mean air pressure (hPa or mb -- equivalent units)
        
Try running auto_regress now with the full list of variables. Do this by copy/pasting the example I already provided to you above, and then just edit the list describing the 'predictors'. Completing this is an important component of your "challenge" to be completed before the next session. 

## Your Challenge 

- Make a time series (line) plot of sim and obs from pred (returned by the auto_regress function)
- Make a bivariate scatter plot of sim *vs* obs. Make sure that this plot includes grid lines and the 1:1 reference line
- Compute the MAE for the regression model
- Compute the SS for the regression model (relative to the climatology). Note that, for convenience, the climatology is one of the columns (called 'clim') in the predicted variable returned by auto_regress. 
- Summarise the results of the regression model in a paragraph of text. Include a description of what physical relationships are described by the coefficients, and comment on the strengths and weaknesses of the model. You may find it helpful to look closely at your scatter plot for this...